## Packages Required

In [10]:
import numpy as np
import scipy.stats as stats
import scipy.optimize as opt
import sklearn
import matplotlib.pyplot as plt

# SVI Jump-Wings Parameterisation

### Parameters

In [30]:
class SVI():
    def __init__(self, a, b, rho, sig, m):
        self.volData = volData
        self.a = a
        self.m = m
        self.b = b
        self.rho = rho
        self.sig = sig

    def SVI_raw(self, volData, a, b, m, rho, sig):
        self.daysExp = volData[0,:]/250
        self.strike = volData[1,:]
        totalVol = a + b * (
            rho * (self.strike - m) + np.sqrt((self.strike - m) ** 2) + sig **2)
        impliedVol = totalVol * self.daysExp
        return impliedVol
    
    def SVI_JW(self, volData):
        self.daysExp = volData[0,:]/250
        self.strike = volData[1,:]
        # raw Parameters
        a = self.a
        b = self.b
        rho = self.rho
        m = self.m
        sig = self.sig
        t = self.daysExp
        
        # SVI Jump-Wings
        wt = (self.a + self.b * ( - self.rho * m + np.sqrt(self.m ** 2 + self.sig ** 2)))
        vt = wt / (t)
        phit = (1 / np.sqrt(wt)) * self.b / 2 * ( - self.m / np.sqrt(self.m ** 2 + self.sig ** 2) + self.rho)
        pt = 1 / np.sqrt(wt) * self.b * (1 - self.rho)
        ct = 1 / np.sqrt(wt) * self.b * ( 1+ self.rho)
        vTilde = (self.a + self.b * self.sig * np.sqrt(1 - self.rho ** 2)) / (t)
        
        # Parameterisation
        beta = self.rho - 2 * phit * np.sqrt(wt) / self.b
        alpha = np.sign(beta) * np.sqrt(1/(beta ** 2) - 1)
        
        bNew = np.sqrt(wt) / 2 * (ct + pt)
        rhoNew = 1 - pt * np.sqrt(wt)/b
        mNew = ((vt - vTilde) * t)/(b *(
            - rho + np.sign(alpha) * np.sqrt(1 + alpha ** 2) - alpha * np.sqrt(1 - rho ** 2)))
        aNew = vTilde * t - b * sig * np.sqrt(1 - rho)        
        if m == 0:
            sigNew = (vt * t - a)/b
        else:
            sigNew = alpha * m
            
        impliedVol = self.SVI_raw(a = aNew, b = bNew, m = mNew, rho = rhoNew, sig = sigNew)
        
        return impliedVol
        

        

In [34]:
def SVI_raw(volData, a, b, m, rho, sig):
        daysExp = volData[0,:]/250
        strike = volData[1,:]
        totalVol = a + b * (
            rho * (strike - m) + np.sqrt((strike - m) ** 2) + sig **2)
        impliedVol = totalVol * daysExp
        return impliedVol
    
def SVI_JW(volData, a, b, m, rho, sig):
        daysExp = volData[0,:]/250
        strike = volData[1,:]
        t = daysExp
        
        # SVI Jump-Wings
        wt = (a + b * ( - rho * m + np.sqrt(m ** 2 + sig ** 2)))
        vt = wt / (t)
        phit = (1 / np.sqrt(wt)) * b / 2 * ( - m / np.sqrt(m ** 2 + sig ** 2) + rho)
        pt = 1 / np.sqrt(wt) * b * (1 - rho)
        ct = 1 / np.sqrt(wt) * b * ( 1+ rho)
        vTilde = (a + b * sig * np.sqrt(1 - rho ** 2)) / (t)
        
        # Parameterisation
        beta = rho - 2 * phit * np.sqrt(wt) / b
        alpha = np.sign(beta) * np.sqrt(1/(beta ** 2) - 1)
        
        bNew = np.sqrt(wt) / 2 * (ct + pt)
        rhoNew = 1 - pt * np.sqrt(wt)/b
        mNew = ((vt - vTilde) * t)/(b *(
            - rho + np.sign(alpha) * np.sqrt(1 + alpha ** 2) - alpha * np.sqrt(1 - rho ** 2)))
        aNew = vTilde * t - b * sig * np.sqrt(1 - rho)        
        if m == 0:
            sigNew = (vt * t - a)/b
        else:
            sigNew = alpha * m
            
        impliedVol = SVI_raw(volData = volData, a = aNew, b = bNew, m = mNew, rho = rhoNew, sig = sigNew)
        
        return impliedVol

## Test

In [4]:
priceData = np.load('joinData.pkl', allow_pickle = True)

In [41]:
allData = priceData.loc['2015-01-02']

yData = allData.filter(['IV'])
yData = np.array(yData).T[0]

xData = allData.filter(['daysExp', 'strike'])
xData = np.array(xData).T


We fit the curve via Least Squares with linear and non-linear contraints on the parameters.


In [76]:
# Constraints
def nonLinConstraintFun(x):
    return (x[0] + x[1] * x[4] * np.sqrt(1 - x[3] **2))

nonLinearConstraint = opt.NonlinearConstraint(nonLinConstraintFun, 0, np.inf)
bnds = ((None, None), (0.0001, None), (None, None), (-0.999, 0.999), (0.0001, None))

#linConstraint = opt.LinearConstraint(np.eye(5),[-np.inf, 0, -np.inf, -0.9999, 0.00001],
                               #      [np.inf, np.inf, np.inf, 0.9999, np.inf])

## Initial Values
initVal = [0.1, 0.1, 0.1, 0.1, 0.1]
cons = ({'type' : 'ineq', 'fun' : nonLinConstraint})
        
## Least Squares Minimisation
funcToMin = lambda x: sum((SVI_JW(xData, x[0], x[1], x[2], x[3], x[4]) - yData)**2)
fittedValues = opt.minimize(funcToMin, initVal, constraints = [nonLinearConstraint], bounds = bnds)
fittedValues

     fun: 12694197.170190187
     jac: array([ 1.23776750e+05,  2.54372919e+08, -1.36155000e+04,  2.31251721e+07,
        3.04737500e+03])
 message: 'Inequality constraints incompatible'
    nfev: 6
     nit: 1
    njev: 1
  status: 4
 success: False
       x: array([0.1, 0.1, 0.1, 0.1, 0.1])

In [75]:
nonLinearConstraint

In [51]:
fittedCurve, _ = opt.curve_fit(SVI_JW, xData, yData, bounds = ([-np.inf, 0, -np.inf, -0.9999, 0.00001], [np.inf, np.inf, np.inf, 0.9999, np.inf]))
fittedCurve

/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:17: RuntimeWarning: invalid value encountered in sqrt
  phit = (1 / np.sqrt(wt)) * b / 2 * ( - m / np.sqrt(m ** 2 + sig ** 2) + rho)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:18: RuntimeWarning: invalid value encountered in sqrt
  pt = 1 / np.sqrt(wt) * b * (1 - rho)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:19: RuntimeWarning: invalid value encountered in sqrt
  ct = 1 / np.sqrt(wt) * b * ( 1+ rho)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:23: RuntimeWarning: invalid value encountered in sqrt
  beta = rho - 2 * phit * np.sqrt(wt) / b
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:26: RuntimeWarning: invalid value encountered in sqrt
  bNew = np.sqrt(wt) / 2 * (ct + pt)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:27: RuntimeWarning: invalid valu

array([-1.18630096,  0.24516435,  2.7264491 , -0.75477657,  0.54781433])

In [52]:
SVI_JW(xData, -1.18630096, 0.24516435, 2.7264491, -0.75377657, 0.54781433)

/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:17: RuntimeWarning: invalid value encountered in sqrt
  phit = (1 / np.sqrt(wt)) * b / 2 * ( - m / np.sqrt(m ** 2 + sig ** 2) + rho)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:18: RuntimeWarning: invalid value encountered in sqrt
  pt = 1 / np.sqrt(wt) * b * (1 - rho)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:19: RuntimeWarning: invalid value encountered in sqrt
  ct = 1 / np.sqrt(wt) * b * ( 1+ rho)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:23: RuntimeWarning: invalid value encountered in sqrt
  beta = rho - 2 * phit * np.sqrt(wt) / b
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:26: RuntimeWarning: invalid value encountered in sqrt
  bNew = np.sqrt(wt) / 2 * (ct + pt)
/var/folders/qz/tbpst7xn3vx136h1lxlk2xhc0000gn/T/ipykernel_66297/84792959.py:27: RuntimeWarning: invalid valu

array([nan, nan, nan, ..., nan, nan, nan])